Імпорт необіхдних бібліотек

In [57]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import itertools
from timeit import timeit

Зафіксуємо seed генератору випадкових чисел

In [58]:
tf.random.set_seed(17)

Викачуємо датасет MNIST, попередньо розділивши його на сукупність для тренування та тесту

In [59]:
(data_train, data_test), data_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

Виводимо інформацію про датасет

In [60]:
print(data_info)

tfds.core.DatasetInfo(
    name='mnist',
    full_name='mnist/3.0.1',
    description="""
    The MNIST database of handwritten digits.
    """,
    homepage='http://yann.lecun.com/exdb/mnist/',
    data_dir='/root/tensorflow_datasets/mnist/3.0.1',
    file_format=tfrecord,
    download_size=11.06 MiB,
    dataset_size=21.00 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
)


Будуємо функцію побудови моделі

In [69]:
def get_tf_model(
    cnn_num_kernels,
    cnn_kernel_size,
    cnn_maxpool,
    nn_size,
    dropout,
    output_num,
    optimizer,
    loss,
    metrics,
):
    """Function for building a sequencial model for MNIST dataset.
    Inputs:
        cnn_num_kernels:    NDArray[k] -- number of kernels for each layer
        cnn_kernel_size:    int        -- size of one side of kernel for each layer
        cnn_maxpool:        int        -- size of one side of maxpool reduction for each layer
        nn_size:            NDArray[n] -- size of each dense layer
        dropout:            float,     -- fraction of connections that will be dropped out
        output_num:         int,       -- number of output classes
        optimizer:          str,       -- optimizer to be used
        loss:               str,       -- loss to be used
        metrics:            list[str]  -- list of metrics to be used
    Output:
        model:              Sequental  -- output model

    """
    # Sequencial model is enough for MNIST
    model = tf.keras.models.Sequential()

    # Convolutional part
    for kernel_num in cnn_num_kernels:
        model.add(
            tf.keras.layers.Conv2D(
                kernel_num,
                (cnn_kernel_size, cnn_kernel_size),
                activation='relu'
            )
        )
        model.add(tf.keras.layers.MaxPooling2D((cnn_maxpool, cnn_maxpool)))

    model.add(tf.keras.layers.Flatten())

    # Dense part
    for size in nn_size:
        model.add(tf.keras.layers.Dense(size, activation='relu'))
        model.add(tf.keras.layers.Dropout(dropout))
    model.add(tf.keras.layers.Dense(output_num, activation='softmax'))

    # Compile model
    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=metrics
    )

    return model

Задаємо batch size, та кількість epoch.

In [62]:
batch_size = 128
epochs = 10

Будуємо навчальний конвеєр:

    1. Переводимо дані з `uint8` [0, 255] у `float32` ([0.0, 1.0])
    2. Зберігаємо дані в кеш перед перетасуванням
    3. Перетасовуємо дані для рівномірного розподілення, для найкращого результату перетасуємо одночасно всі зображення.
    4. Розділяємо перетасований датасет на пакети (batches) фіксованого розміру для пакетного стохастичного градієнтного спуску
    5. Попередньо завантажуємо дані для кращої продуктивності

Функції запозичено з [прикладу для Keras](https://www.tensorflow.org/datasets/keras_example)




In [63]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  assert image.dtype == tf.uint8
  return tf.cast(image, tf.float32) / 255., label

data_train = data_train.map(
    normalize_img,
    num_parallel_calls=tf.data.AUTOTUNE
)
data_train = data_train.cache()
data_train = data_train.shuffle(data_info.splits['train'].num_examples)
data_train = data_train.batch(batch_size)
data_train = data_train.prefetch(tf.data.AUTOTUNE)

Будуємо тестувальний конвеєр.

Для нього вже не потрібно перетасовувати дані.

In [64]:
data_test = data_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
data_test = data_test.batch(batch_size)
data_test = data_test.cache()
data_test = data_test.prefetch(tf.data.AUTOTUNE)


Задаємо параметри, що потрібно змінювати при підборі

In [65]:
cnn_num_kernels = [
    [8],
    [16],
    [32],
    [8, 16],
    [16, 32],
    [32, 64],
    [8, 16, 16],
    [16, 32, 32],
    [32, 64, 64]
]
cnn_kernel_size = [3, 5, 7]
cnn_maxpool = [2, 4]
nn_size = [
    [64],
    [128],
    [256],
    [64, 128],
    [128, 256],
    [256, 128],
    [128, 64]
]

Задаємо фіксовані параметри

In [66]:
dropout = 0.5
output_num = 10
optimizer = 'adam'
loss = 'sparse_categorical_crossentropy'
metrics = ['sparse_categorical_accuracy']

Спершу шукаємо параметри нейроної мережі без згорткових шарів.

In [73]:
param_opt = None
model_opt = None
best_accuracy = 0.0

for size in nn_size:
    print('Parameters:', size)

    model = get_tf_model(
        [],
        None,
        None,
        size,
        dropout=dropout,
        output_num=output_num,
        optimizer=optimizer,
        loss=loss,
        metrics=metrics
    )

    model.fit(
        data_train,
        epochs=epochs,
        batch_size=batch_size,
        verbose=0
    )

    accuracy = model.evaluate(data_test, verbose=2)[1]
    if accuracy > best_accuracy:
        model_opt = tf.keras.models.clone_model(model)
        param_opt = size
        best_accuracy = accuracy

    print()

print('Optimal parameters accuracy:', param_opt)
print('Best accuracy:', best_accuracy)
print(model_opt.summary())

Parameters: [64]
79/79 - 0s - loss: 0.1349 - sparse_categorical_accuracy: 0.9609 - 253ms/epoch - 3ms/step

Parameters: [128]
79/79 - 0s - loss: 0.0864 - sparse_categorical_accuracy: 0.9744 - 254ms/epoch - 3ms/step

Parameters: [256]
79/79 - 1s - loss: 0.0675 - sparse_categorical_accuracy: 0.9785 - 546ms/epoch - 7ms/step

Parameters: [64, 128]
79/79 - 0s - loss: 0.1255 - sparse_categorical_accuracy: 0.9628 - 244ms/epoch - 3ms/step

Parameters: [128, 256]
79/79 - 0s - loss: 0.0894 - sparse_categorical_accuracy: 0.9734 - 279ms/epoch - 4ms/step

Parameters: [256, 128]
79/79 - 0s - loss: 0.0742 - sparse_categorical_accuracy: 0.9779 - 317ms/epoch - 4ms/step

Parameters: [128, 64]
79/79 - 0s - loss: 0.1013 - sparse_categorical_accuracy: 0.9706 - 279ms/epoch - 4ms/step

Optimal parameters accuracy: [256]
Best accuracy: 0.9785000085830688
Model: "sequential_34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 fl

Шукаємо серед множини параметрів той, що дасть найкращу тестову точність і зберігаємо його.

Паралельно шукаємо набір параметрів, що дасть нам саме оптимальне відношення точності на одиницю продуктивності.

In [42]:
param_opt = None
model_opt = None
best_accuracy = 0.0

# Let us track accuracy / perfrormance ratio
param_opt_perf = None
model_opt_perf = None
exec_time = None
best_acc_perf_ratio = 0.0

# Perform cartesian product of four lists
for param in itertools.product(cnn_num_kernels, cnn_kernel_size, cnn_maxpool, nn_size):

    print('Parameters:', param)

    model = get_tf_model(
        cnn_num_kernels=param[0],
        cnn_kernel_size=param[1],
        cnn_maxpool=param[2],
        nn_size=param[3],
        dropout=dropout,
        output_num=output_num,
        optimizer=optimizer,
        loss=loss,
        metrics=metrics,
    )

    model.fit(
        data_train,
        epochs=epochs,
        batch_size=batch_size,
        verbose=0
    )

    accuracy = model.evaluate(data_test, verbose=2)[1]

    # In practice, 30 is enough for Law of Large Numbers
    time_test = lambda : model.predict(np.zeros((1, 28, 28, 1)), verbose=0)
    exec_time = timeit(time_test, number=30) / 30

    acc_perf_ratio = accuracy / exec_time

    if accuracy > best_accuracy:
        model_opt = tf.keras.models.clone_model(model)
        param_opt = param
        best_accuracy = accuracy

    if acc_perf_ratio > best_acc_perf_ratio:
        model_opt_perf = tf.keras.models.clone_model(model)
        param_opt_perf = param
        best_acc_perf_ratio = acc_perf_ratio

    print()

print('Optimal parameters accuracy:', param_opt)
print('Best accuracy:', best_accuracy)
print(model_opt.summary())
model_opt.save_weights('./model_opt')

print('Optimal parameters accuracy / performance ratio:', param_opt)
print('Best accuracy per performance:', best_accuracy)
print(model_opt.summary())
model_opt.save_weights('./model_opt_perf')

Parameters: ([8], 3, 2, [64])
79/79 - 1s - loss: 0.0555 - sparse_categorical_accuracy: 0.9810 - 999ms/epoch - 13ms/step
Parameters: ([8], 3, 2, [128])
79/79 - 1s - loss: 0.0458 - sparse_categorical_accuracy: 0.9849 - 864ms/epoch - 11ms/step
Parameters: ([8], 3, 2, [256])
79/79 - 1s - loss: 0.0399 - sparse_categorical_accuracy: 0.9857 - 930ms/epoch - 12ms/step
Parameters: ([8], 3, 2, [64, 64])


KeyboardInterrupt: 